# TP1 - Regresión
Notebook inicial para el trabajo práctico.

In [51]:
# 1. Importación de librerías necesarias
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler   # u otros scalers
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, LassoCV, RidgeCV, ElasticNetCV
from sklearn.metrics import mean_squared_error, r2_score

import holidays

In [38]:
# 2. Carga de datos. Se carga el dataset que contiene los viajes de taxi.
file_path= 'uber_fares.csv'
df = pd.read_csv(file_path)

# visualizacion de algunos datos
df.head()

,key,date,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,24238194,2015-05-07 19:52:06.0000003,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,27835199,2009-07-17 20:04:56.0000002,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,44984355,2009-08-24 21:45:00.00000061,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1
3,25894730,2009-06-26 08:22:21.0000001,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3
4,17610152,2014-08-28 17:47:00.000000188,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5


#### Contexto  
El proyecto trata sobre **Uber Inc.**, la compañía de taxis más grande del mundo. En este trabajo, nuestro objetivo es **predecir la tarifa de futuros viajes**.  

Uber brinda servicio a millones de clientes cada día, por lo que gestionar adecuadamente sus datos es clave para desarrollar nuevas estrategias de negocio y obtener mejores resultados.  

### Variables del conjunto de datos  

**Variables explicativas:**  
- **key**: identificador único de cada viaje.  
- **pickup_datetime**: fecha y hora en que se inició el viaje.  
- **passenger_count**: cantidad de pasajeros en el vehículo (dato ingresado por el conductor).  
- **pickup_longitude**: longitud del punto de inicio del viaje.  
- **pickup_latitude**: latitud del punto de inicio del viaje.  
- **dropoff_longitude**: longitud del punto de destino.  
- **dropoff_latitude**: latitud del punto de destino.  

**Variable objetivo (target):**  
- **fare_amount**: costo del viaje en dólares.  

In [ ]:
# 3. Análisis descriptivo de las variables
# Comentario: Se realiza un análisis inicial para comprender el comportamiento de cada variable.
# - Descripción estadística (df.describe())
# - Visualización de valores únicos y rangos


# Columnas, ¿cuáles son variables numéricas y cuales variables categóricas?
df.columns


Index(['key', 'date', 'fare_amount', 'pickup_datetime', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count'],
      dtype='object')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   key                200000 non-null  int64  
 1   date               200000 non-null  object 
 2   fare_amount        200000 non-null  float64
 3   pickup_datetime    200000 non-null  object 
 4   pickup_longitude   200000 non-null  float64
 5   pickup_latitude    200000 non-null  float64
 6   dropoff_longitude  199999 non-null  float64
 7   dropoff_latitude   199999 non-null  float64
 8   passenger_count    200000 non-null  int64  
dtypes: float64(5), int64(2), object(2)
memory usage: 13.7+ MB


In [8]:
df.apply(pd.unique)

key                  [24238194, 27835199, 44984355, 25894730, 17610...
date                 [2015-05-07 19:52:06.0000003, 2009-07-17 20:04...
fare_amount          [7.5, 7.7, 12.9, 5.3, 16.0, 4.9, 24.5, 2.5, 9....
pickup_datetime      [2015-05-07 19:52:06 UTC, 2009-07-17 20:04:56 ...
pickup_longitude     [-73.99981689453125, -73.994355, -74.005043, -...
pickup_latitude      [40.73835372924805, 40.728225, 40.74077, 40.79...
dropoff_longitude    [-73.99951171875, -73.99471, -73.962565, -73.9...
dropoff_latitude     [40.72321701049805, 40.750325, 40.772647, 40.8...
passenger_count                             [1, 3, 5, 2, 4, 6, 0, 208]
dtype: object

In [9]:
df.nunique()

key                  200000
date                 200000
fare_amount            1244
pickup_datetime      196629
pickup_longitude      71066
pickup_latitude       83835
dropoff_longitude     76894
dropoff_latitude      90585
passenger_count           8
dtype: int64

In [39]:
# Eliminar la columna 'key' porque no aporta al análisis
df = df.drop(columns=['key'])

In [32]:
df['pickup_datetime'].isna().sum()

np.int64(0)

In [40]:
# Eliminar la columna 'date' porque redunda con el dato de 'pickup_datetime'
df = df.drop(columns=['date'])

In [ ]:
df['passenger_count'].value_counts()

passenger_count
1      138425
2       29428
5       14009
3        8881
4        4276
6        4271
0         709
208         1
Name: count, dtype: int64

In [41]:
# Eliminar filas donde passenger_count sea igual a 208
df = df[df['passenger_count'] != 208]

In [ ]:
# PARA ENCONTRAR VALORES NEGATIVOS QUE NO CORRESPONDEN Y MINIMOS ABSURDOS
df['fare_amount'].value_counts().sort_index().head(20)

fare_amount
-52.00      2
-50.50      1
-49.57      1
-23.70      1
-10.90      1
-7.30       1
-6.90       1
-5.70       1
-5.50       1
-5.00       1
-3.50       3
-3.30       1
-3.00       2
 0.00       5
 0.01       2
 0.11       1
 2.50     825
 2.60       1
 2.90     478
 3.00     355
Name: count, dtype: int64

Eliminamos los siguientes datos considerados erroneos:
- Fare 0
- Fare 0.01 y 0.11 tambien eliminar ya que no se corresponden a la distancia recorrida
- Todos los fare negativos tambien se eliminan

In [42]:
# Eliminar filas donde fare_amount sea menor o igual a 0.11
df = df[df['fare_amount'] > 0.11]

In [26]:
# Verificar si hay valores NaN en fare__amount
df['fare_amount'].isna().sum()

np.int64(0)

Para lograr determinar si un fare amount grande es un outlier necesitamos calcular la variable distancia para lograr comprender si se corresponden los valores o no. Luego de una evaluación de la mejor función de distancia, llegamos a la conclusión que la manhattan se ajusta mejor a la estructura de la ciudad en estudio.

In [43]:
# Distancia Manhattan
def distancia_absoluta(lat1, lon1, lat2, lon2):
    """
    Calcula la suma de las diferencias absolutas entre latitud y longitud.
    Es decir, la distancia Manhattan en coordenadas.
    """
    return np.abs(lat2 - lat1) + np.abs(lon2 - lon1)

# Creamos al columna distancia
df["distancia_recorrida"] = distancia_absoluta(
    df["pickup_latitude"], df["pickup_longitude"],
    df["dropoff_latitude"], df["dropoff_longitude"]
)

# Distancia Manhattan en kilómetros (aproximado)
df["distancia_recorrida_km"] = df["distancia_recorrida"] * 111

In [44]:
df['fare_amount'].value_counts().sort_index().tail(20)

fare_amount
163.00    1
165.00    1
165.33    1
170.00    1
171.35    1
180.00    1
183.00    1
190.00    1
190.83    1
191.80    1
192.33    1
196.00    1
200.00    1
206.38    1
220.00    1
230.00    1
250.00    1
275.00    1
350.00    1
499.00    1
Name: count, dtype: int64

In [45]:
df.nlargest(20, 'fare_amount')[['fare_amount', 'distancia_recorrida', 'distancia_recorrida_km']]

,fare_amount,distancia_recorrida,distancia_recorrida_km
170081,499.00,0.000011,0.001221
4292,350.00,0.000000,0.000000
185325,275.00,0.000000,0.000000
71715,250.00,0.000000,0.000000
197493,230.00,0.501812,55.701132
29261,220.00,0.035656,3.957816
23682,206.38,0.858774,95.323914
196647,200.00,0.000000,0.000000
184901,196.00,0.000017,0.001887
33911,192.33,0.000000,0.000000


In [46]:
# Eliminar viajes con distancia recorrida menor a 0.1 km
df = df[df['distancia_recorrida_km'] >= 0.1]

In [34]:
df.sort_values('fare_amount', ascending=False)

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,dist_absoluta,distancia_recorrida,distancia_recorrida_km
197493,230.00,2014-09-07 08:39:00 UTC,-73.937765,40.758267,-74.382200,40.700890,2,0.501812,0.501812,55.701132
29261,220.00,2013-09-27 12:36:11 UTC,-73.801147,40.671653,-73.790402,40.646742,1,0.035656,0.035656,3.957816
23682,206.38,2014-11-02 22:40:20 UTC,-73.782161,40.644601,-74.449079,40.836457,1,0.858774,0.858774,95.323914
39716,191.80,2012-06-26 22:28:00 UTC,-73.822978,40.788500,-73.548110,41.045037,1,0.531405,0.531405,58.985955
114282,190.83,2015-04-04 23:59:34 UTC,-73.781746,40.644630,-74.320007,40.716450,1,0.610081,0.610081,67.718960
...,...,...,...,...,...,...,...,...,...,...
47911,2.50,2012-06-24 19:41:15 UTC,-73.985996,40.743394,-73.985372,40.744158,1,0.001388,0.001388,0.154068
147851,2.50,2015-06-16 10:32:26 UTC,-73.969704,40.755325,-73.968933,40.755070,1,0.001026,0.001026,0.113903
12432,2.50,2014-03-26 13:31:11 UTC,-73.954993,40.765505,0.000000,0.000000,1,114.720498,114.720498,12733.975278
40290,2.50,2013-10-16 18:14:00 UTC,0.000000,0.000000,-74.043428,40.788097,6,114.831525,114.831525,12746.299275


In [48]:
# Eliminar columnas de pickup y dropoff (latitud y longitud)
df = df.drop(columns=[
    'pickup_latitude', 'pickup_longitude',
    'dropoff_latitude', 'dropoff_longitude'
])

In [49]:
# Resetear los índices del DataFrame después de los filtrados
df = df.reset_index(drop=True)
df

,fare_amount,pickup_datetime,passenger_count,distancia_recorrida,distancia_recorrida_km
0,7.5,2015-05-07 19:52:06 UTC,1,0.015442,1.714050
1,7.7,2009-07-17 20:04:56 UTC,1,0.022455,2.492505
2,12.9,2009-08-24 21:45:00 UTC,1,0.074355,8.253405
3,5.3,2009-06-26 08:22:21 UTC,3,0.023313,2.587743
4,16.0,2014-08-28 17:47:00 UTC,5,0.065221,7.239531
...,...,...,...,...,...
193360,3.0,2012-10-28 10:49:00 UTC,1,0.001447,0.160617
193361,7.5,2014-03-14 01:09:00 UTC,1,0.024733,2.745363
193362,30.9,2009-06-29 00:42:00 UTC,2,0.190959,21.196449
193363,14.5,2015-05-20 14:56:25 UTC,1,0.043945,4.877930


In [54]:
# Crear objeto de feriados de New York
feriados = holidays.US(state='NY')

# Si pickup_datetime es string, primero conviértelo a datetime
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])

# Crear la variable dummy: 1 si es feriado, 0 si no
df['es_feriado'] = df['pickup_datetime'].dt.date.apply(lambda x: 1 if x in feriados else 0)

# Crear variable dummy: 1 si es fin de semana (sábado o domingo), 0 si es día de semana (lunes a viernes)
df['es_fin_de_semana'] = df['pickup_datetime'].dt.weekday.apply(lambda x: 1 if x >= 5 else 0)

# Crear variable: 1 si es fin de semana o feriado, 0 si es día de semana y no feriado
df['es_no_laborable'] = ((df['es_fin_de_semana'] == 1) | (df['es_feriado'] == 1)).astype(int)

# Dropear columnas auxiliares
df = df.drop(columns=['es_feriado', 'es_fin_de_semana'])

df

,fare_amount,pickup_datetime,passenger_count,distancia_recorrida,distancia_recorrida_km,es_no_laborable
0,7.5,2015-05-07 19:52:06+00:00,1,0.015442,1.714050,0
1,7.7,2009-07-17 20:04:56+00:00,1,0.022455,2.492505,0
2,12.9,2009-08-24 21:45:00+00:00,1,0.074355,8.253405,0
3,5.3,2009-06-26 08:22:21+00:00,3,0.023313,2.587743,0
4,16.0,2014-08-28 17:47:00+00:00,5,0.065221,7.239531,0
...,...,...,...,...,...,...
193360,3.0,2012-10-28 10:49:00+00:00,1,0.001447,0.160617,1
193361,7.5,2014-03-14 01:09:00+00:00,1,0.024733,2.745363,0
193362,30.9,2009-06-29 00:42:00+00:00,2,0.190959,21.196449,0
193363,14.5,2015-05-20 14:56:25+00:00,1,0.043945,4.877930,0


# Ingeniería de Variables para Análisis de Viajes en Taxi

## 1. Creación de Variables Derivadas

### Distancia entre Origen y Destino
- **Distancia Euclidiana**: Calculada en grados entre dos puntos. Es rápida de calcular, aunque no representa la ruta real sobre la superficie terrestre.
- **Distancia Haversine**: Calculada en kilómetros o metros, es una medida más precisa para coordenadas geográficas, considerando la curvatura de la Tierra.
- **Distancia Absoluta (Manhattan)**: Suma de las diferencias absolutas de latitud y longitud, útil para aproximar trayectos en ciudades con calles en cuadrícula (como Manhattan).

### Características Temporales del Viaje
Extraídas a partir de `pickup_datetime`:
- **Hora del día** (`0-23`): Permite analizar si el horario influye en la tarifa.
- **Día de la semana** (`Lunes` a `Domingo`): Útil para detectar diferencias entre días laborables y fines de semana.
- **Semana del año**: Puede ayudar a detectar patrones estacionales o eventos especiales.
- **Mes** o **Estación del año**: Para analizar estacionalidad en la demanda.
- **¿Es hora pico?**: Variable booleana según franjas horarias (ej.: 7-9 AM y 5-7 PM).
- **¿Es fin de semana?**: Variable booleana para diferenciar sábados y domingos.

### Duración del Viaje
- Si se dispone del tiempo de llegada, calcular la duración real.
- Si no, **estimar la duración** utilizando la distancia y una velocidad promedio (ejemplo: duración = distancia / velocidad promedio).

### Variables de Interacción
- **Distancia × Cantidad de Pasajeros**: Para observar si viajes largos con más pasajeros modifican la tarifa por persona.
- **Hora × Día de la semana**: Permite captar efectos combinados entre horario y tipo de día.

---

## 2. Identificación y Tratamiento de Datos Problemáticos

### Valores Atípicos (Outliers)
- Detectar valores extremos en variables como distancia, tarifa o cantidad de pasajeros.
- Analizar si tienen sentido físico (ej.: viajes con distancia cero pero tarifa alta).

### Datos Faltantes (Missing Values)
- Identificar columnas con valores faltantes.
- Decidir el tratamiento adecuado: imputación, eliminación, o reemplazo según el caso.

---

### Notas
- Las variables temporales pueden codificarse como **cualitativas ordinales** para ciertos modelos (ej.: día de la semana, semana del año).
- La ingeniería de variables adecuada puede mejorar significativamente el rendimiento de los modelos de regresión y la interpretación de los resultados.

In [22]:
def distancia_euclidiana(lat1, lon1, lat2, lon2):
    """
    Calcula la distancia euclidiana entre dos puntos (lat1, lon1) y (lat2, lon2).
    La medida es en grados, porque no considera la curvatura de la tierra.
    """
    return np.sqrt((lat2 - lat1) ** 2 + (lon2 - lon1) ** 2)

# Distancia Manhattan
def distancia_absoluta(lat1, lon1, lat2, lon2):
    """
    Calcula la suma de las diferencias absolutas entre latitud y longitud.
    Es decir, la distancia Manhattan en coordenadas.
    """
    return np.abs(lat2 - lat1) + np.abs(lon2 - lon1)

In [23]:
df["dist_euclidiana"] = distancia_euclidiana(
    df["pickup_latitude"], df["pickup_longitude"],
    df["dropoff_latitude"], df["dropoff_longitude"]
)
df["dist_absoluta"] = distancia_absoluta(
    df["pickup_latitude"], df["pickup_longitude"],
    df["dropoff_latitude"], df["dropoff_longitude"]
)

In [14]:
print(df.head())  # Muestra las primeras 5 filas de todo el df
print(df.tail())  # Muestra las últimas 5 filas

        key                           date  fare_amount  \
0  24238194    2015-05-07 19:52:06.0000003          7.5   
1  27835199    2009-07-17 20:04:56.0000002          7.7   
2  44984355   2009-08-24 21:45:00.00000061         12.9   
3  25894730    2009-06-26 08:22:21.0000001          5.3   
4  17610152  2014-08-28 17:47:00.000000188         16.0   

           pickup_datetime  pickup_longitude  pickup_latitude  \
0  2015-05-07 19:52:06 UTC        -73.999817        40.738354   
1  2009-07-17 20:04:56 UTC        -73.994355        40.728225   
2  2009-08-24 21:45:00 UTC        -74.005043        40.740770   
3  2009-06-26 08:22:21 UTC        -73.976124        40.790844   
4  2014-08-28 17:47:00 UTC        -73.925023        40.744085   

   dropoff_longitude  dropoff_latitude  passenger_count  dist_euclidiana  \
0         -73.999512         40.723217                1         0.015140   
1         -73.994710         40.750325                1         0.022103   
2         -73.962565      

In [13]:
df[["dist_euclidiana", "dist_absoluta", "fare_amount"]].head(10)

,dist_euclidiana,dist_absoluta,fare_amount
0,0.015140,0.015442,7.5
1,0.022103,0.022455,7.7
2,0.053109,0.074355,12.9
3,0.016528,0.023313,5.3
4,0.051031,0.065221,16.0
5,0.000000,0.000000,4.9
6,0.120825,0.170584,24.5
7,0.000000,0.000000,2.5
8,0.027623,0.029763,9.7
9,0.050496,0.071250,12.5


In [16]:
print(df[["dist_euclidiana", "dist_absoluta"]].describe())

       dist_euclidiana  dist_absoluta
count    199999.000000  199999.000000
mean          0.256968       0.328972
std           9.216476      11.627817
min           0.000000       0.000000
25%           0.012432       0.015863
50%           0.021489       0.027697
75%           0.038335       0.050330
max        2434.714983    3270.155787


In [25]:
# Distancia haversine (en km)
def haversine(lat1, lon1, lat2, lon2):
    # Radio de la Tierra en km
    R = 6371
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi/2)**2 + np.cos(phi1)*np.cos(phi2)*np.sin(dlambda/2)**2
    return 2*R*np.arcsin(np.sqrt(a))

In [27]:
df['dist_haversine'] = haversine(
    df['pickup_latitude'], df['pickup_longitude'],
    df['dropoff_latitude'], df['dropoff_longitude'])

In [24]:
df[["dist_euclidiana", "dist_absoluta", "dist_haversine", "fare_amount"]].head(10)

KeyError: "['dist_haversine'] not in index"

In [25]:
df[df['fare_amount'] == 0.01][['fare_amount', 'dist_absoluta']]

,fare_amount,dist_absoluta
21658,0.01,0.000023
191228,0.01,0.098550


In [26]:
df[df['fare_amount'] == 0.11][['fare_amount', 'dist_absoluta']]

,fare_amount,dist_absoluta
149342,0.11,0.000032


In [27]:
df[df['fare_amount'] == 0][['fare_amount', 'dist_absoluta']]

,fare_amount,dist_absoluta
20744,0.0,0.000000
22182,0.0,0.103810
87467,0.0,114.776978
156738,0.0,0.000008
197172,0.0,0.000000


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   key                200000 non-null  int64  
 1   date               200000 non-null  object 
 2   fare_amount        200000 non-null  float64
 3   pickup_datetime    200000 non-null  object 
 4   pickup_longitude   200000 non-null  float64
 5   pickup_latitude    200000 non-null  float64
 6   dropoff_longitude  199999 non-null  float64
 7   dropoff_latitude   199999 non-null  float64
 8   passenger_count    200000 non-null  int64  
dtypes: float64(5), int64(2), object(2)
memory usage: 13.7+ MB


In [ ]:
# 4. Análisis y tratamiento de datos faltantes
# Comentario: Se identifican valores faltantes y se decide cómo tratarlos (eliminación, imputación, etc.).
# - df.isnull().sum()
# - Justificación de la decisión tomada

In [ ]:
# 5. Análisis y tratamiento de datos atípicos (outliers)
# Comentario: Se detectan y tratan valores atípicos en las variables numéricas.
# - Visualización con boxplots
# - Decisión sobre el tratamiento (eliminación, ajuste, etc.)

In [ ]:
# 6. Visualización de datos
# Comentario: Se grafican histogramas y scatterplots para analizar la distribución y relaciones entre variables.
# - Histograma de cada variable
# - Scatterplots entre variables relevantes
# - Diagramas de caja

In [ ]:
# 7. Codificación de variables categóricas (si corresponde)
# Comentario: Se codifican variables categóricas para su uso en modelos (OneHotEncoder, LabelEncoder, etc.).
# - Ejemplo: df['var'] = ...

In [ ]:
# 8. Matriz de correlación
# Comentario: Se calcula la matriz de correlación para analizar dependencias entre variables.
# - sns.heatmap(df.corr(), annot=True)

In [ ]:
# 9. Estandarización o escalado de datos
# Comentario: Se aplican técnicas de escalado para mejorar el desempeño de los modelos.
# - StandardScaler, MinMaxScaler, etc.

In [ ]:
# 10. División de datos: Train-Test (y opcionalmente Validación)
# Comentario: Se divide el conjunto de datos en entrenamiento y prueba.
# - from sklearn.model_selection import train_test_split
# - X_train, X_test, y_train, y_test = train_test_split(...)

X_train, X_test, y_train, y_test = train_test_split(df.drop(columns='fare_amount'), df['fare_amount'], test_size=0.2, random_state=42)

In [ ]:
X_train.describe()

In [ ]:
# 11. Implementación de modelos de regresión
# 11.a. Regresión lineal múltiple (LinearRegression)
# Comentario: Se ajusta el modelo base de regresión lineal.

In [ ]:
# 11.b. Métodos de gradiente descendente (SGDRegressor, otros)
# Comentario: Se prueban diferentes variantes de gradiente descendente.
# - Se grafican errores vs iteraciones (loss vs epochs)

In [ ]:
# 11.c. Métodos de regularización (Lasso, Ridge, Elastic Net)
# Comentario: Se aplican modelos con regularización y se varía el coeficiente.
# - Se comparan métricas para distintos valores de alpha.

In [ ]:
# 12. Evaluación de modelos: métricas de regresión
# Comentario: Se calculan las métricas elegidas para train y test (R2, MSE, RMSE, MAE, MAPE).
# - Justificación de las métricas utilizadas

In [ ]:
# 13. Gráficos de residuos
# Comentario: Se grafican los residuos para analizar el ajuste de los modelos.


In [ ]:
# 14. Análisis de fitting y conclusiones intermedias
# Comentario: Se analiza el fitting del modelo y se discuten los efectos de la regularización y gradiente descendente.


In [ ]:
# 15. Optimización y comparación de hiperparámetros
# Comentario: Se varían los hiperparámetros y se observa el efecto en el desempeño del modelo.
# - GridSearchCV, RandomizedSearchCV, o análisis manual


In [ ]:
# 16. Comparación de modelos
# Comentario: Se compara el desempeño de los distintos modelos y se selecciona el mejor.
# - Justificación de la métrica de comparación

In [ ]:
# 17. Conclusiones finales
# Comentario: Se redacta una conclusión sobre el trabajo realizado, principales hallazgos y aprendizajes.
